In [1]:
from IPython.core.display import display, HTML
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important; } ' 
        '.container { width:100% !important; } '
        '.end_space { min-height:0px !important; } '
    '</style>'
))

In [2]:
import findspark
findspark.init()
findspark.find()

'D:\\Softwares\\Spark\\spark-2.4.7-bin-hadoop2.7'

In [3]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

conf = SparkConf().setAppName('appName').setMaster('local')
sc = SparkContext(conf= conf)

In [4]:
spark = SparkSession(sc).builder.appName('basics').getOrCreate()

In [11]:
#read text data as rdd
lines = sc.textFile("./Datasets/ml-100k/u.data")
type(lines)

pyspark.rdd.RDD

In [7]:
#create another RDD from lines with only ratings
ratings = lines.map(lambda x : x.split()[2])

In [12]:
#actually perform action on RDD. takes time to execute.
#this returns a dictionary?? -- returns a collection
results = ratings.countByValue()
type(results)

collections.defaultdict

In [23]:
import collections
for i,j in collections.OrderedDict(sorted(results.items())).items():
    print(f"{i} * rating - {j} numbers")

1 * rating - 6110 numbers
2 * rating - 11370 numbers
3 * rating - 27145 numbers
4 * rating - 34174 numbers
5 * rating - 21201 numbers


## FakeFirendz

In [5]:
ff = sc.textFile("./Datasets/fakefriends.csv")

In [32]:
def parse_fake(x):
    line = x.split(",")
    age, friends = line[2], int(line[3].strip())
    return age, friends

In [33]:
rdd = ff.map(parse_fake)

In [34]:
rdd.collect()

[('33', 385),
 ('26', 2),
 ('55', 221),
 ('40', 465),
 ('68', 21),
 ('59', 318),
 ('37', 220),
 ('54', 307),
 ('38', 380),
 ('27', 181),
 ('53', 191),
 ('57', 372),
 ('54', 253),
 ('56', 444),
 ('43', 49),
 ('36', 49),
 ('22', 323),
 ('35', 13),
 ('45', 455),
 ('60', 246),
 ('67', 220),
 ('19', 268),
 ('30', 72),
 ('51', 271),
 ('25', 1),
 ('21', 445),
 ('22', 100),
 ('42', 363),
 ('49', 476),
 ('48', 364),
 ('50', 175),
 ('39', 161),
 ('26', 281),
 ('53', 197),
 ('43', 249),
 ('27', 305),
 ('32', 81),
 ('58', 21),
 ('64', 65),
 ('31', 192),
 ('52', 413),
 ('67', 167),
 ('54', 75),
 ('58', 345),
 ('35', 244),
 ('52', 77),
 ('25', 96),
 ('24', 49),
 ('20', 1),
 ('40', 254),
 ('51', 283),
 ('36', 212),
 ('19', 269),
 ('62', 31),
 ('19', 5),
 ('41', 278),
 ('44', 194),
 ('57', 294),
 ('59', 158),
 ('59', 284),
 ('20', 100),
 ('62', 442),
 ('69', 9),
 ('58', 54),
 ('31', 15),
 ('52', 169),
 ('21', 477),
 ('48', 135),
 ('33', 74),
 ('30', 204),
 ('52', 393),
 ('45', 184),
 ('22', 179),
 ('2

In [44]:
sumByAge = rdd.mapValues(lambda x : (x,1)).reduceByKey(lambda x,y : (x[0] + y[0], x[1] + y[1]))

In [47]:
avgByAge = sumByAge.mapValues(lambda x : round(x[0]/x[1],0))

In [52]:
sorted(avgByAge.collect())

[('18', 343.0),
 ('19', 213.0),
 ('20', 165.0),
 ('21', 351.0),
 ('22', 206.0),
 ('23', 246.0),
 ('24', 234.0),
 ('25', 197.0),
 ('26', 242.0),
 ('27', 228.0),
 ('28', 209.0),
 ('29', 216.0),
 ('30', 236.0),
 ('31', 267.0),
 ('32', 208.0),
 ('33', 325.0),
 ('34', 246.0),
 ('35', 212.0),
 ('36', 247.0),
 ('37', 249.0),
 ('38', 194.0),
 ('39', 169.0),
 ('40', 251.0),
 ('41', 269.0),
 ('42', 304.0),
 ('43', 231.0),
 ('44', 282.0),
 ('45', 310.0),
 ('46', 224.0),
 ('47', 233.0),
 ('48', 281.0),
 ('49', 185.0),
 ('50', 255.0),
 ('51', 302.0),
 ('52', 341.0),
 ('53', 223.0),
 ('54', 278.0),
 ('55', 296.0),
 ('56', 307.0),
 ('57', 259.0),
 ('58', 117.0),
 ('59', 220.0),
 ('60', 203.0),
 ('61', 256.0),
 ('62', 221.0),
 ('63', 384.0),
 ('64', 281.0),
 ('65', 298.0),
 ('66', 276.0),
 ('67', 215.0),
 ('68', 270.0),
 ('69', 235.0)]

In [62]:
age_x = [int(x[0]) for x in sorted(avgByAge.collect())]
num_f = [int(x[1]) for x in sorted(avgByAge.collect())]

In [71]:
new_df = pd.DataFrame({"Age":age_x, "Avg_Frns":num_f})
new_df.plot(x='Age', y='Avg_Frns')

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

In [18]:
import pandas as pd
df= pd.read_csv("./Datasets/fakefriends.csv", names=["idx","Name","Age","f_count"], index_col=0)
df.head()

,Name,Age,f_count
idx,,,
0,Will,33,385
1,Jean-Luc,26,2
2,Hugh,55,221
3,Deanna,40,465
4,Quark,68,21


In [22]:
with open("./Datasets/fakefriends.csv", "r") as fp:
    lines = fp.readlines()
    fp.close()

In [31]:
[w.strip() for w in lines[0].split(",")][2:]

['33', '385']

# Understanding ReduceByKey

In [78]:
data = [('Project', 10),
('Gutenberg’s', 1),
('Alice’s', 1),
('Adventures', 1),
('in', 1),
('Wonderland', 1),
('Project', 1),
('Gutenberg’s', 1),
('Adventures', 1),
('in', 1),
('Wonderland', 1),
('Project', 1),
('Gutenberg’s', 1)]

type(data)

list

In [79]:
rdd = sc.parallelize(data)

In [80]:
type(rdd)

pyspark.rdd.RDD

In [81]:
op = rdd.reduceByKey(lambda x,y : x+y)

In [82]:
for i in op.collect():
    print(i)

('Project', 12)
('Gutenberg’s', 3)
('Alice’s', 1)
('Adventures', 2)
('in', 2)
('Wonderland', 2)
